<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import QuantLib as ql
import prettytable as pt
from MachineTrading.Instruments.fixed_rate import BTP

In [41]:
coupon, maturityDate, issueDate = (0.90, ql.Date(1, 4, 2031), ql.Date(1, 10, 2020))

eval_date = ql.Date(6, 5, 2022)
base_yld = 3.04
base_prc = 83.55

In [42]:
ql.Settings.instance().evaluationDate = eval_date

In [43]:
bondYield = base_yld / 100.0

In [53]:
settlementDays = 2
faceAmount = 100.0

tenor = ql.Period(ql.Semiannual)
calendar = ql.NullCalendar()
convention = ql.Unadjusted
terminationDateConvention = convention
rule = ql.DateGeneration.Backward
endOfMonth = True

schedule = ql.Schedule(issueDate, maturityDate, tenor, calendar, convention, terminationDateConvention, rule, endOfMonth)
coupons = ql.DoubleVector(1)
coupons[0] = coupon / 100.0

accrualDayCounter = ql.ActualActual(ql.ActualActual.Bond, schedule)
paymentConvention = ql.ModifiedFollowing
bond = ql.FixedRateBond(settlementDays, faceAmount, schedule, coupons, accrualDayCounter, paymentConvention)

curve = ql.FlatForward(eval_date, 
                       ql.QuoteHandle(ql.SimpleQuote(bondYield)), 
                       ql.ActualActual(ql.ActualActual.Bond), 
                       ql.Compounded)
handle = ql.YieldTermStructureHandle(curve)
handle.enableExtrapolation()
bondEngine = ql.DiscountingBondEngine(handle)    
bond.setPricingEngine(bondEngine)

cleanPrice = bond.cleanPrice()
dirtyPrice = bond.dirtyPrice()
accruedAmount = bond.accruedAmount()

tab = pt.PrettyTable(['item', 'QuantLib', 'Expected'])
tab.add_row(['clean price', cleanPrice, base_prc])
tab.add_row(['dirty price', dirtyPrice, base_prc+0.08391])
tab.add_row(['accrued amount', accruedAmount, 0.08391])
tab.float_format = '.4'

print(tab)

+----------------+----------+----------+
|      item      | QuantLib | Expected |
+----------------+----------+----------+
|  clean price   | 83.5527  | 83.5500  |
|  dirty price   | 83.6437  | 83.6339  |
| accrued amount |  0.0910  |  0.0839  |
+----------------+----------+----------+
